In [4]:
import pandas as pd
import numpy as np
import psycopg2
from decimal import Decimal
from psycopg2.extras import RealDictCursor

In [2]:
def connect_to_db():
    return psycopg2.connect(
        host="localhost",
        user="postgres",
        password="rayyan123",
        database="dummy_db",
        port=5432
    )


In [3]:
db = connect_to_db()
print('Connected Successfully')

Connected Successfully


In [10]:
# creating a broker
cursor = db.cursor()
cursor

<cursor object at 0x0000013ACF24AF80; closed: 0>

In [11]:
"select count(*) as total_suppliers from suppliers"

'select count(*) as total_suppliers from suppliers'

In [12]:
cursor.execute("select count(*) as total_suppliers from suppliers")

In [13]:
row = cursor.fetchone()
print(row)

(50,)


In [14]:
list(row)[0]   # converting to list and fetching 0th index

50

In [22]:
queries = {
    "Total Suppliers": "SELECT COUNT(*) AS total_suppliers FROM suppliers",

    "Total Products": "SELECT COUNT(*) AS total_products FROM products",

    "Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS total_categories FROM products",

    "Total Sale Value (Last 3 Months)": """
        SELECT ROUND(SUM(ABS(se.change_quantity) * p.price)::numeric, 2) AS total_sales_value_in_last_3_months
        FROM stock_entries se
        JOIN products p ON p.product_id = se.product_id
        WHERE se.change_type = 'Sale'
        AND se.entry_date >= (
            SELECT MAX(entry_date) - INTERVAL '3 months'
            FROM stock_entries
        )
    """,

    "Total Restock Value (Last 3 Months)": """
        SELECT ROUND(SUM(ABS(se.change_quantity) * p.price)::numeric, 2) AS total_restock_value_in_last_3_months
        FROM stock_entries se
        JOIN products p ON p.product_id = se.product_id
        WHERE se.change_type = 'Restock'
        AND se.entry_date >= (
            SELECT MAX(entry_date) - INTERVAL '3 months'
            FROM stock_entries
        )
    """,

    "Below Reorder & No Pending Reorders": """
        SELECT COUNT(*) AS products_needing_restock
        FROM products p
        WHERE p.stock_quantity < p.reorder_level
        AND p.product_id NOT IN (
            SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending'
        )
    """
}


In [23]:
result={}
for label, query in queries.items():
    cursor.execute(query)
    row= cursor.fetchone()
    value=list(row)[0]
    if isinstance(value, Decimal):
        value = float(value)
    result[label] = value
    

In [24]:
result

{'Total Suppliers': 50,
 'Total Products': 201,
 'Total Categories Dealing': 5,
 'Total Sale Value (Last 3 Months)': None,
 'Total Restock Value (Last 3 Months)': 9999.0,
 'Below Reorder & No Pending Reorders': 14}

In [25]:
def get_basic_info(cursor):
    queries = {
        "Total Suppliers": "SELECT COUNT(*) AS total_suppliers FROM suppliers",

        "Total Products": "SELECT COUNT(*) AS total_products FROM products",

        "Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS total_categories FROM products",

        "Total Sale Value (Last 3 Months)": """
            SELECT ROUND(SUM(ABS(se.change_quantity) * p.price)::numeric, 2) AS total_sales_value_in_last_3_months
            FROM stock_entries se
            JOIN products p ON p.product_id = se.product_id
            WHERE se.change_type = 'Sale'
              AND se.entry_date >= (
                SELECT MAX(entry_date) - INTERVAL '3 months'
                FROM stock_entries
            )
        """,

        "Total Restock Value (Last 3 Months)": """
            SELECT ROUND(SUM(ABS(se.change_quantity) * p.price)::numeric, 2) AS total_restock_value_in_last_3_months
            FROM stock_entries se
            JOIN products p ON p.product_id = se.product_id
            WHERE se.change_type = 'Restock'
              AND se.entry_date >= (
                SELECT MAX(entry_date) - INTERVAL '3 months'
                FROM stock_entries
            )
        """,

        "Below Reorder & No Pending Reorders": """
            SELECT COUNT(*) AS products_needing_restock
            FROM products p
            WHERE p.stock_quantity < p.reorder_level
              AND p.product_id NOT IN (
                  SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending'
              )
        """
    }

    results = {}
    for label, query in queries.items():
        cursor.execute(query)
        row = cursor.fetchone()   # tuple like (value,)
        value = row[0]

        # Convert Decimal → float for JSON/dashboard compatibility
        if isinstance(value, Decimal):
            value = float(value)

        results[label] = value

    return results


In [26]:
queries = {
        "Suppliers Contact Details": "SELECT supplier_name, contact_name, email, phone FROM suppliers",

        "Products with Supplier and Stock": """
            SELECT 
                p.product_name,
                s.supplier_name,
                p.stock_quantity,
                p.reorder_level
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            ORDER BY p.product_name ASC
        """,

        "Products Needing Reorder": """
            SELECT product_name, stock_quantity, reorder_level
            FROM products
            WHERE stock_quantity <= reorder_level
        """
    }

tables = {}
for label, query in queries.items():
    cursor.execute(query)
    tables[label] = cursor.fetchall()

In [27]:
tables

{'Suppliers Contact Details': [('Anderson-Thompson',
   'Bonnie Davis',
   'zacharysanchez@hotmail.com',
   '829.485.9853x0522'),
  ('Rowland Ltd', 'Beth Stevens', 'joshua60@yahoo.com', '(779)942-0726'),
  ('Baxter-Meadows',
   'Lisa Lewis',
   'andersonchristina@yahoo.com',
   '449-766-7325'),
  ('Wilson, Graham and Williams',
   'David Martinez',
   'abrown@hotmail.com',
   '+1-653-827-5215x266'),
  ('Smith, Kennedy and Moreno',
   'Victoria Gonzalez',
   'mary30@williams-moore.com',
   '891-859-2775x35297'),
  ('Middleton LLC',
   'Megan Miller',
   'kelseywilliams@gmail.com',
   '001-724-731-6199x4596'),
  ('Evans Inc',
   'Danielle Moore',
   'ihoffman@warren.com',
   '+1-951-447-1975x770'),
  ('Lawrence, Garcia and Hernandez',
   'Tamara Johnson',
   'georgeherrera@hotmail.com',
   '+1-635-460-8476x270'),
  ('Young, Browning and Ware',
   'Heather Hill',
   'stephensjason@yahoo.com',
   '(514)361-6411x489'),
  ('Newton, Valencia and Carr',
   'Kimberly Collins',
   'jonathanjohns

In [28]:
def get_additonal_tables(cursor):
    queries = {
        "Suppliers Contact Details": """
            SELECT supplier_name, contact_name, email, phone
            FROM suppliers
        """,

        "Products with Supplier and Stock": """
            SELECT 
                p.product_name,
                s.supplier_name,
                p.stock_quantity,
                p.reorder_level
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            ORDER BY p.product_name ASC
        """,

        "Products Needing Reorder": """
            SELECT product_name, stock_quantity, reorder_level
            FROM products
            WHERE stock_quantity <= reorder_level
        """
    }

    tables = {}
    for label, query in queries.items():
        cursor.execute(query)
        tables[label] = cursor.fetchall()

    return tables


In [29]:
get_additonal_tables(cursor)

{'Suppliers Contact Details': [('Anderson-Thompson',
   'Bonnie Davis',
   'zacharysanchez@hotmail.com',
   '829.485.9853x0522'),
  ('Rowland Ltd', 'Beth Stevens', 'joshua60@yahoo.com', '(779)942-0726'),
  ('Baxter-Meadows',
   'Lisa Lewis',
   'andersonchristina@yahoo.com',
   '449-766-7325'),
  ('Wilson, Graham and Williams',
   'David Martinez',
   'abrown@hotmail.com',
   '+1-653-827-5215x266'),
  ('Smith, Kennedy and Moreno',
   'Victoria Gonzalez',
   'mary30@williams-moore.com',
   '891-859-2775x35297'),
  ('Middleton LLC',
   'Megan Miller',
   'kelseywilliams@gmail.com',
   '001-724-731-6199x4596'),
  ('Evans Inc',
   'Danielle Moore',
   'ihoffman@warren.com',
   '+1-951-447-1975x770'),
  ('Lawrence, Garcia and Hernandez',
   'Tamara Johnson',
   'georgeherrera@hotmail.com',
   '+1-635-460-8476x270'),
  ('Young, Browning and Ware',
   'Heather Hill',
   'stephensjason@yahoo.com',
   '(514)361-6411x489'),
  ('Newton, Valencia and Carr',
   'Kimberly Collins',
   'jonathanjohns

In [17]:
def add_new_manual_id(cursor, db, p_name , p_category , p_price , p_stock , p_reorder, p_supplier):
    proc_call= "call AddNewProductManualID(%s, %s, %s ,%s ,%s, %s)"
    params= (p_name , p_category , p_price , p_stock , p_reorder, p_supplier)
    cursor.execute(proc_call, params)
    db.commit()

In [18]:
def get_categories(cursor1):
    cursor1.execute("select Distinct category  from products  order by category  asc")
    rows= cursor1.fetchall()
    return [row["category"] for row in rows]

In [21]:
cursor1 = db.cursor(cursor_factory=RealDictCursor)
cursor1

<cursor object at 0x0000013ACF7A68A0; closed: 0>

In [22]:
get_categories(cursor1)

['Clothing', 'Electronics', 'Furniture', 'Groceries', 'Toys']

In [29]:
def get_suppliers(cursor1):
    cursor1.execute("select supplier_id , supplier_name from suppliers order by  supplier_name asc")
    return cursor1.fetchall()

In [30]:
suppliers={}
suppliers= get_suppliers(cursor1)


In [31]:
supplier_ids=[s["supplier_id"] for s in suppliers]
supplier_names=[s["supplier_name"] for s in suppliers]

In [33]:
supplier_names

['Anderson-Thompson',
 'Armstrong-Vance',
 'Barker, White and Carson',
 'Barrett Ltd',
 'Baxter-Meadows',
 'Charles Inc',
 'Clark Group',
 'Douglas Ltd',
 'Elliott-Ayers',
 'Evans Inc',
 'Franklin, Kane and Price',
 'Freeman-Gordon',
 'Gallagher-Miller',
 'Gomez PLC',
 'Hall-Brown',
 'Harris-Cummings',
 'Henderson LLC',
 'Hensley-Branch',
 'Hudson Inc',
 'Johnson-Bass',
 'Kaufman Ltd',
 'Lawrence, Garcia and Hernandez',
 'Lloyd and Sons',
 'Mann-Marshall',
 'Mendoza-Jones',
 'Middleton LLC',
 'Miller-Martinez',
 'Moody-Vang',
 'Morgan-Andrews',
 'Morgan Inc',
 'Moss-Evans',
 'Newton, Valencia and Carr',
 'Ortega-Mahoney',
 'Patrick, Walter and Harrison',
 'Perez, Price and Wallace',
 'Reynolds-Phillips',
 'Rogers-Greene',
 'Rowe PLC',
 'Rowland Ltd',
 'Smith, Kennedy and Moreno',
 'Stewart, Williams and Cox',
 'Taylor-Love',
 'Tran LLC',
 'Tucker-Arnold',
 'Turner-Davis',
 'Vega, Cook and Miller',
 'Williams Ltd',
 'Wilson, Graham and Williams',
 'Wong Group',
 'Young, Browning and War

In [34]:
print(lambda x: supplier_names[supplier_ids.index(x)])

<function <lambda> at 0x0000013ACF85C400>


In [35]:
def get_all_products(cursor):
    cursor.execute("select product_id, product_name from products order by  product_name")
    return cursor.fetchall()

def get_product_history(cursor, product_id):
    query ="select * from product_inventory_history where product_id= %s order by record_date Desc"
    cursor.execute(query , (product_id,))
    return cursor.fetchall()

In [37]:
products = get_all_products(cursor1)
products

[RealDictRow([('product_id', 13), ('product_name', 'Ability Snack')]),
 RealDictRow([('product_id', 64), ('product_name', 'Account Toy')]),
 RealDictRow([('product_id', 69), ('product_name', 'Actually Toy')]),
 RealDictRow([('product_id', 48), ('product_name', 'After Table')]),
 RealDictRow([('product_id', 112), ('product_name', 'Against Table')]),
 RealDictRow([('product_id', 81), ('product_name', 'Alone Toy')]),
 RealDictRow([('product_id', 87), ('product_name', 'Already Snack')]),
 RealDictRow([('product_id', 40), ('product_name', 'Among Toy')]),
 RealDictRow([('product_id', 11), ('product_name', 'Amount Snack')]),
 RealDictRow([('product_id', 2), ('product_name', 'And Table')]),
 RealDictRow([('product_id', 105), ('product_name', 'Another Device')]),
 RealDictRow([('product_id', 129), ('product_name', 'Appear Toy')]),
 RealDictRow([('product_id', 179), ('product_name', 'As Toy')]),
 RealDictRow([('product_id', 182), ('product_name', 'Ask Table')]),
 RealDictRow([('product_id', 178)

In [38]:
history = get_product_history(cursor, 9)
history

[(9, 'Stock Entry', datetime.datetime(2025, 5, 20, 0, 0), 54, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 5, 17, 0, 0), -31, 'Sale', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 4, 30, 0, 0), 45, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 4, 9, 0, 0), 24, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 4, 9, 0, 0), -42, 'Sale', 48),
 (9, 'Shipment', datetime.datetime(2025, 3, 13, 0, 0), 178, None, 48),
 (9, 'Stock Entry', datetime.datetime(2025, 3, 9, 0, 0), 30, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 3, 3, 0, 0), -19, 'Sale', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 3, 1, 0, 0), 95, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 2, 26, 0, 0), 81, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 2, 23, 0, 0), 6, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 2, 6, 0, 0), 37, 'Restock', 48),
 (9, 'Stock Entry', datetime.datetime(2025, 1, 21, 0, 0), 26, 'Restock', 48),
 (9, '

In [ ]:
def get_pending_reorders(cursor):
    cursor.execute("""
    select r.reorder_id , p.product_name
    from reorders as r join products as p 
    on r.product_id= p.product_id
    """)
    return cursor.fetchall()

In [42]:
get_pending_reorders(cursor1)

[RealDictRow([('reorder_id', 1), ('product_name', 'Someone Shirt')]),
 RealDictRow([('reorder_id', 3), ('product_name', 'Space Toy')]),
 RealDictRow([('reorder_id', 4), ('product_name', 'Blue Device')]),
 RealDictRow([('reorder_id', 5), ('product_name', 'Mouth Shirt')]),
 RealDictRow([('reorder_id', 6), ('product_name', 'School Table')]),
 RealDictRow([('reorder_id', 7), ('product_name', 'Four Shirt')]),
 RealDictRow([('reorder_id', 8), ('product_name', 'Fast Shirt')]),
 RealDictRow([('reorder_id', 9), ('product_name', 'Character Table')]),
 RealDictRow([('reorder_id', 10), ('product_name', 'Fact Device')]),
 RealDictRow([('reorder_id', 11), ('product_name', 'Return Table')]),
 RealDictRow([('reorder_id', 12), ('product_name', 'Scene Table')]),
 RealDictRow([('reorder_id', 13), ('product_name', 'Mission Snack')]),
 RealDictRow([('reorder_id', 14), ('product_name', 'Old Shirt')]),
 RealDictRow([('reorder_id', 15), ('product_name', 'Within Toy')]),
 RealDictRow([('reorder_id', 16), ('pro